In [112]:
import json
import pandas as pd
import re

from pathlib import Path

In [113]:
path = Path("./results")
save_path = Path("./results/multinli_transforms_dev_matched.jsonl")
file_paths = path.glob("*.json")

In [114]:
### Weird stuff:
### \xa0 = \u00a0 instead of spaces
### Numbers on separate lines
### Multi-line answers
### Double answer (1. ... 2. ... OR 1. ... 2...)
### Notes about the transformation that took place


def split_content(content, pair_id):
    content = content.strip()
    splits = re.split(f'\n[ \n]*', content)
    s1 = splits[0][3:].strip()
    s2 = splits[1][3:].strip()

    return s1, s2

In [115]:
result_dict = {}

for file_path in file_paths:
    with file_path.open() as f:
        content = json.load(f)['choices'][0]['message']['content']
    pair_id, sent_num, tst_style = file_path.stem.split('_')
    if not result_dict.get(pair_id):
        result_dict[pair_id] = {"style": tst_style}
    result_dict[pair_id][f"{sent_num}_1"], result_dict[pair_id][f"{sent_num}_2"] = split_content(content, pair_id)

In [116]:
result_list = []
for key, value in result_dict.items():
    result_list.append(
        {
            "pairID": key,
            "style": value['style'],
            "sentence1_transform1": value['sen1_1'],
            "sentence1_transform2": value['sen1_2'],
            "sentence2_transform1": value['sen2_1'],
            "sentence2_transform2": value['sen2_2'],
        }
    )

df = pd.DataFrame(result_list)
df.to_json(save_path, orient="records", lines=True)